In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

D:\Documenti\Anaconda\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [2]:
def sigmoid(x):
    return 1.0/(1+ np.exp(-x))

def sigmoid_derivative(x):
    return x * (1.0 - x)

In [3]:
#class for a NN with one hidden layer with 4 neurons (?)
class NeuralNetwork:
    #initialize neural network
    def __init__(self,x,y,n):
        #public:
        self.input = x
        self.y     = y
        
        #private:
        self.__neurons  = n
        self.__output   = np.zeros([1,1])
        self.__weights1 = np.random.rand(self.__neurons,self.input.shape[0])
        self.__weights2 = np.random.rand(1,self.__neurons)
    
    #reset neural network
    def reset(self):
        self.__init__(self.input,self.y)
        
    def getoutput(self):
        return self.__output
        
    #feedforward with zero bias (b2=0)
    def feedforward(self):
        
        self.layer1   = sigmoid(np.dot(self.__weights1 , self.input))
        self.__output = sigmoid(np.dot(self.__weights2 , self.layer1))
    
    #backpropagation algorithm (Andrew NG)
    def backpropagation(self,l_reg):
        #compute "error" for each layer
        d_weights_output = (self.y - self.__output) 
        d_weights_hidden = np.multiply(np.dot(self.__weights2.T , d_weights_output) , np.multiply(self.layer1 , (1-self.layer1)))
        d_weights_input  = np.multiply(np.dot(self.__weights1.T , d_weights_hidden) , np.multiply(self.input , (1-self.input)))
        
        #update weights
        self.__weights1 = np.add(self.__weights1 , 0.05 * np.kron(d_weights_hidden , self.input.T)) + l_reg * self.__weights1
        self.__weights2 = np.add(self.__weights2 , 0.05 * np.kron(d_weights_output , self.layer1.T)) + l_reg * self.__weights2

In [4]:
## input
df = pd.read_csv("C:\\Users\\unieuro\\Documents\\ML\\WondrousMachineLaundry-master\\titanic\\data\\train.csv")

## mapping strings to numbers
df["Sex"] = df["Sex"].replace({"male":0, "female":1})
df["Embarked"] = df["Embarked"].replace({'C':0, 'S':1, 'Q':2})

## handling names
title_dict = {"Mr.":0, "Mrs.":1, "Mme.":1, "Miss.":2, "Mlle.":2, "Master.":3, "Dr.":4, "Rev.": 5}
for i_row, row in df.iterrows():
    name_str = row["Name"]
    if not isinstance(name_str, str):
        break # break the loop if the names have already been replaced by ints
    df.loc[i_row,"Name"] = 6 # assign the default value at the begining
    for word in name_str.split():
        if word in title_dict.keys():
            df.loc[i_row,"Name"] = title_dict[word]
            break # break the loop if the key value has been found
        
## handling nans
df = df.drop(columns=["Cabin"])
df["Age"] = df["Age"].fillna(df["Age"].median())
df["Embarked"] = df["Embarked"].fillna(df["Embarked"].mode().iloc[0])

## get dummies
features = ["Pclass", "Name"]
for f in features:
    df = pd.concat([df.drop(columns=f), pd.get_dummies(df[f], prefix=f)], axis=1)

## creating input and output dataframes
yf = df["Survived"]
Xf = df.drop(columns=["PassengerId", "Survived", "Ticket"])

## normalizing
features = ["Age", "Fare"]
Xf[features] = (Xf[features] - Xf[features].mean()) / Xf[features].std()
print(Xf.head())

   Sex       Age  SibSp  Parch      Fare  Embarked  Pclass_1  Pclass_2  \
0    0 -0.565419      1      0 -0.502163       1.0         0         0   
1    1  0.663488      1      0  0.786404       0.0         1         0   
2    1 -0.258192      0      0 -0.488580       1.0         0         0   
3    1  0.433068      1      0  0.420494       1.0         1         0   
4    0  0.433068      0      0 -0.486064       1.0         0         0   

   Pclass_3  Name_0  Name_1  Name_2  Name_3  Name_4  Name_5  Name_6  
0         1       1       0       0       0       0       0       0  
1         0       0       1       0       0       0       0       0  
2         1       0       0       1       0       0       0       0  
3         0       0       1       0       0       0       0       0  
4         1       1       0       0       0       0       0       0  


In [5]:
Xdata=Xf.loc[:,:].to_numpy()
ydata=yf.loc[:].to_numpy()

In [32]:
n = 4
l_reg =  0.01 / len(ydata)


train_frac=0.8
Ltrain = np.zeros(int(len(ydata)*0.8))

nn = NeuralNetwork(np.asmatrix(Xdata[0]).T,np.asmatrix(ydata[0]),n)
nn.feedforward()
nn.backpropagation(l_reg)
Ltrain[0] = nn.getoutput()

for i in range(len(Ltrain)):
    nn.input = np.asmatrix(Xdata[i]).T
    nn.y = np.asmatrix(ydata[i]).T
    for j in range(100):
        nn.feedforward()
        nn.backpropagation(l_reg)
    Ltrain[i] = nn.getoutput()

In [33]:
length = len(Ltrain)
Ltest = np.zeros(len(ydata)-length)

for i in range(len(ydata)-length):
    nn.input = np.asmatrix(Xdata[i+length]).T
    nn.y = np.asmatrix(ydata[i+length]).T
    for j in range(100):
        nn.feedforward()
    Ltest[i] = nn.getoutput()

In [34]:
ltest = np.zeros(len(Ltest))
for i in range(len(Ltest)):
    if (Ltest[i]>0.5):
        ltest[i] = 1
    else:
        ltest[i]=0

In [35]:
from sklearn.metrics import f1_score
f1_score(ydata[length:].T,ltest.T)

0.7256637168141593

In [36]:
ltrain = np.zeros(len(Ltrain))
for i in range(len(Ltrain)):
    if (Ltrain[i]>0.5):
        ltrain[i] = 1
    else:
        ltrain[i]=0

In [37]:
from sklearn.metrics import f1_score
f1_score(ydata[:length].T,ltrain.T)

0.9854545454545456